In [ ]:
from matplotlib.image import imread
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
sess = sagemaker.Session()

In [ ]:
from PIL import Image
images_dir = "../Resources/train/NORMAL/"
train_path = "../Resources/train/NORMAL/"

Image.open(images_dir + "IM-0115-0001.jpeg")

In [3]:
from pathlib import Path
import os, shutil
from os import listdir
import numpy as np
import cv2

# Image folder
images_dir
WIDTH = 1858//4
LENGTH = 2090//4
CHANNELS = 3

dim = (WIDTH, LENGTH)

# Resizing all the images to same dimension
X_image_train = []
for fname in listdir(images_dir):
    fpath = os.path.join(images_dir, fname)
    im = Image.open(fpath)
    im_resized = im.resize(dim)
    X_image_train.append(im_resized)

## Converting the image to numpy array
X_image_array=[]
for x in range(len(X_image_train)):
    X_image=np.array(X_image_train[x],dtype='uint8')
    img2 = cv2.merge((X_image,X_image,X_image))
    X_image_array.append(img2)

# Checking the size of a single image
X_image_array[15].shape



(522, 464, 3)

In [4]:
del X_image_train

In [5]:
final_tensor = np.stack(X_image_array)
final_tensor.shape

(1341, 522, 464, 3)

In [ ]:
# final_tensor = final_tensor.reshape(1341, LENGTH, WIDTH, CHANNELS)

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(rotation_range = 20,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              rescale=1/255,
                              featurewise_center = True,
                              shear_range=0.1,
                              zoom_range=.1,
                              horizontal_flip=True,
                              fill_mode="nearest")

image_gen.fit(final_tensor)
del final_tensor

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss",mode="min",patience=2)

In [8]:
train_path = "../Resources/train/"
test_path = "../Resources/val/"
batch_size=8
image_shape = (WIDTH, LENGTH, CHANNELS)



train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                               color_mode="rgb",
                                               batch_size=batch_size,
                                               class_mode="binary")

test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode="rgb",
                                               batch_size=batch_size,
                                               class_mode="binary",
                                              shuffle=False)


Found 4929 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [9]:
train_image_gen.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [10]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15165451658845838267,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16229677608512480614
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5753587537827171161
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11338085172
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6087616996711810473
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"]

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizer import Adam

model= Sequential()

model.add(Conv2D(filters=96, kernel_size=(12,12), strides=(4,4), input_shape=image_shape, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
          
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation="relu", padding="same"))
model.add(BatchNormalization())
          
model.add(Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation="relu", padding="same"))
model.add(BatchNormalization())
          
model.add(Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation="relu", padding="same"))
model.add(BatchNormalization())
          
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Flatten())
    
model.add(Dense(2096, activation="relu"))
model.add(Dropout(0.5))
          
model.add(Dense(2096, activation="relu"))
model.add(Dropout(0.5))
          
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate = 0.001)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 114, 128, 96)      41568     
_________________________________________________________________
batch_normalization (BatchNo (None, 114, 128, 96)      384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 63, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 63, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 63, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 31, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 31, 384)       8

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])

results = model.fit_generator(train_image_gen, epochs=20,
                             validation_data=test_image_gen,
                             callbacks=[early_stop])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 617 steps, validate for 2 steps
Epoch 1/20
617/617 [==============================] - 306s 495ms/step - loss: 2.5415 - accuracy: 0.8328 - val_loss: 38.8627 - val_accuracy: 0.5000
Epoch 2/20
249/617 [===========>..................] - ETA: 2:59 - loss: 0.5361 - accuracy: 0.8635

In [ ]:
pred = model.predict_generator(test_image_gen)

predictions = pred>0.5

predictions

from sklearn.metrics import classification_report, confusion_matrix

test_image_gen.classes

print(train_image_gen.class_indices)
print("\n")
print(classification_report(test_image_gen.classes,predictions))

print(confusion_matrix(test_image_gen.classes, predictions))